In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys 
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.common.exceptions import ElementNotInteractableException
from chromedriver_py import binary_path  

import os
import sys
import pandas as pd
import time

'''
options = Options()
options.page_load_strategy = 'normal'
options.add_argument("--start-maximized")
options.add_argument("user-data-dir=/tmp/storedLoginInformation3")
prefs = {'download.prompt_for_download' : False}
options.add_experimental_option('prefs', prefs)

service = Service()


driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
'''

'\noptions = Options()\noptions.page_load_strategy = \'normal\'\noptions.add_argument("--start-maximized")\noptions.add_argument("user-data-dir=/tmp/storedLoginInformation3")\nprefs = {\'download.prompt_for_download\' : False}\noptions.add_experimental_option(\'prefs\', prefs)\n\nservice = Service()\n\n\ndriver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)\n'

In [2]:
from geography.classes.UserClass import UserClass
from classes.LoginClass import Login
from classes.LoginClass import PasswordManager


pm = PasswordManager()
if pm.password is None:
    password = pm.get_password()
    print("Password set!")
else: pass



#this causes browser window to open again which I suppose is fine
#but it probably means there's some header stuff (driver bit??) in LoginClass that I can get rid of
# restart kernel some time, comment out that bit in LoginCLass, run this again.

#USER INFORMATION: User and Basin Information Setup (sSet this for yourself and the current basin)
external_user = False
basin_code = "cngo" #testing new search terms for amzn, cldo, danu, rgna, yukn
master_user = "selena"

Password set!


In [3]:
currentUser = UserClass(basin_code, master_user, external_user)
currentUser.getName()
paths = currentUser.getPath("excel")

#Base Paths 
user_name = paths["user_name"]
geography_folder = paths["geography_folder"]
download_folder_temp = paths["download_folder_temp"]
download_folder = paths["download_folder"]
status_file = paths["status_file"]

#instantiate init login
login = Login(user_name = user_name, password=password)
login._init_login()

selena
False


NameError: name 'driver' is not defined

In [8]:
#and the search terms sheet
#df = pd.read_excel('TrackingSheet_basinterms.xlsx')
df = pd.read_excel('TrackingSheet_NEWbasinterms.xlsx') #to test the new basin search terms
#the sheet is in the same folder as this notebook, otherwise would need full file path
#in NoLinkClass we'd want to refer to basin_code variable in main.py
row = df[df['BCODE'] == basin_code.upper()]
search_term = row['Basin_Specific_Terms'].values[0]
#print(search_term)

In [11]:
'''
def login_tufts_user(user_name):
    time.sleep(3)
    print("Logging in user with userName " + user_name)
    driver.get("https://login.ezproxy.library.tufts.edu/login?auth=tufts&url=http://www.nexisuni.com")
    time.sleep(3)
    driver.find_element(By.CSS_SELECTOR, ".btn-shib > .login").click()
    time.sleep(3)
    driver.find_element(By.ID, "username").send_keys(user_name)
    driver.find_element(By.ID, "password").send_keys(password) #this line used to say ...send_keys("")
    
    #And then have a
    #time.sleep(5)
    #that would wait for the user to type in their password.

    try:
        driver.find_element(By.NAME, "_eventId_proceed").click()
        time.sleep(5)
    except NoSuchElementException:  
        print("Logged In _eventId_proceed")

    try:
        driver.find_element(By.ID, "trust-browser-button").click()
        time.sleep(5)
    except NoSuchElementException:  
        print("Logged In trust-browser-button")
    time.sleep(5)

    return driver
    '''




In [12]:
from classes.NoLinkClass import NoLinkClass
nlc = NoLinkClass(driver, search_term)
# the steps below assume we're already on nexis uni homepage.
# so the below line can be added in case we're not logging in
driver.get("https://advance-lexis-com.ezproxy.library.tufts.edu/bisacademicresearchhome?crid=d975d3e5-053d-44cc-85d9-4a1b2f583bea&pdmfid=1516831&pdisurlapi=true")

In [13]:
nlc = NoLinkClass(driver, search_term)
nlc.get_search_term()
nlc._init_search()
nlc._box_1()
nlc._box_2()
nlc._box_3()
nlc._box_4()
nlc.complete_search()

Search Term: Congo OR Zaire OR Lualaba OR Lubudi OR Lomami OR Lulonga OR Ikelemba OR Ruki OR Kasai OR Lufira OR Luvua OR Lukuga OR Luama OR Elila OR Ulindi OR Lowa OR Maiko OR Lindi OR Aruwimi OR Itimbiri OR Mongala OR Ubangi OR Sangha OR Likouala OR Alima OR Lefini OR Kwango OR Cuango OR Kwenge OR Kwilu OR Loange OR Lukeni OR Lopori OR Maringa OR Tele OR Ngoko OR Motaba OR Giri


Initializing search with driver: <selenium.webdriver.chrome.webdriver.WebDriver (session="1b756159bac22fb25ad8ab92f362e6a4")> and search term: Congo OR Zaire OR Lualaba OR Lubudi OR Lomami OR Lulonga OR Ikelemba OR Ruki OR Kasai OR Lufira OR Luvua OR Lukuga OR Luama OR Elila OR Ulindi OR Lowa OR Maiko OR Lindi OR Aruwimi OR Itimbiri OR Mongala OR Ubangi OR Sangha OR Likouala OR Alima OR Lefini OR Kwango OR Cuango OR Kwenge OR Kwilu OR Loange OR Lukeni OR Lopori OR Maringa OR Tele OR Ngoko OR Motaba OR Giri


In [ ]:
# maybe add the 2 cells above to NoLinkClass as its own class called Search or somethin
# then next steps for me will be parsing out main.py to create more task-specific classes
# like downloading

In [ ]:
#adding everything below as modified from main.py...
#would it be better to have these all defined in a class somehwere else?

In [8]:
#Set Basin Status CSV File 
status_data = pd.read_csv(status_file, index_col=0)

def single_basin_search():
    print("MAIN: Starting a single basin search for ", basin_code)
    
    for index, row in status_data.iterrows():
        #basin = row['Basin_Name']
        basin = row['BCODE'] 
        start_date = row['start_date']
        end_date = row['end_date']
        finished = row['finished']

        if finished == 2:
             print(basin_code, " is done!")
             time.sleep(1)
             quit()

        if finished != 1:

            #STEP 2: Set Date Range
            change_date(search_term, start_date, end_date)

            #STEP 3: Group Duplicates
            group_duplicates()

            #STEP 4: Set Sort by to Date (oldest to Newest)
            set_sort_by_date()

            #STEP 5: Get Current Result Count
            result_count = get_result_count()
            result_count_int = int(result_count)

            if result_count_int < 1000:

                #STEP 6: Download Excel 
                download_outcome = download_excel(basin_code, 1, result_count)

                #STEP 7: Mark complete
                if download_outcome == True:
                    file_min = "result_min_1" + "_"
                    file_max = "result_max_" + result_count + "_"
                    file_start_date = "start_date_" + start_date + "_"
                    file_end_date = "end_date_" + end_date
                    
                    final_file_name = "ResultsList_" + basin_code + "_202207_" + file_min + file_max + file_start_date + file_end_date

                    update_status_success(index, result_count, final_file_name)
                else: 
                    update_status_failure(index, result_count)

                print("MAIN: Finished the Basin")
                time.sleep(12)

            else: 
                update_status_over_limit(index, result_count)

        else:     
            print("The basin ", basin, " from ", start_date, " to ", end_date, " is already done so we are skipping")
            time.sleep(1)

#STEP 2: Set Date Range
def change_date(search_link, start_date, end_date):
    print("STEP 2: Set Date Range")
    print("Setting the date range from ", start_date, " to ", end_date)
    driver.execute_script("window.scrollTo(0,120)")
    time.sleep(2)
    driver.execute_script("window.scrollTo(0,400)")
    time.sleep(2)

    #Step 1: Check if Min Val is Visibile (will be when the timeline button is clicked)
    attempt_to_open_timeline()

    set_min_date(search_link, start_date)
    set_max_date(search_link, end_date)

    driver.find_element(By.CSS_SELECTOR, ".save").click()
    time.sleep(4)
    print("STEP 2: Finished")

#Function 2A: Open the Timeline Button 
def attempt_to_open_timeline():
    print("Step 2A: Attempt to Open Timeline we will scroll to and try to click the button")
    timeline_opened = check_timeline_opened()

    #if the timeline is not opened then open it 
    if timeline_opened == False:
        #TO DO: Handle when this doesn't open
        open_timeline_button()

#Function 2B: Set Min date 
def set_min_date(search_link, start_date):
    print("Starting set_min_date")
    print(start_date)
    min_count = 0

    #Try to open the timeline window if it does not open get the page again to reset 
    while min_count < 5:
        try:
            min_val_button = driver.find_element(By.CSS_SELECTOR, ".min-val")   
            time.sleep(1)  
            min_count = 10 
        except NoSuchElementException:  
            print("NoSuchElementException couldn't find min value ", min_count)
            driver.get(search_link)
            time.sleep(4)

            attempt_to_open_timeline()
                    
            min_count = min_count + 1
        
    #Clear out the current date 
    for x in range(12):
        min_val_button.click()
        min_val_button.send_keys(Keys.BACKSPACE)
        time.sleep(.1)

    #Put the new date in
    min_val_button.send_keys(start_date)
    print("Min date set")
    time.sleep(2)

#Function 2C: Set Max date 
def set_max_date(search_link, end_date):
    print("Starting set_max_date ")
    print(end_date)
    max_count = 0

    try:
        driver.find_element(By.CSS_SELECTOR, ".max-val").click()
        time.sleep(1)   
    except NoSuchElementException:  
        print("NoSuchElementException couldn't find max value ", max_count)
        driver.get(search_link)
        time.sleep(4) 
        attempt_to_open_timeline()
        max_count = max_count + 1
        
 
    for x in range(12):
        driver.find_element(By.CSS_SELECTOR, ".max-val").send_keys(Keys.BACKSPACE)
        time.sleep(.2)   
    time.sleep(1)
 
    driver.find_element(By.CSS_SELECTOR, ".max-val").send_keys(end_date)
    print("Max date set")

    time.sleep(2)

#Function 2D: Open the Timeline Button
def open_timeline_button():
    timeline_opened = check_timeline_opened()
    #print("Check timeline status")
    #print(timeline_opened)

    if timeline_opened == False:
        print("The timeline is closed so we will open the timeline")

        #Open the Timeline
        try:
            driver.find_element(By.ID, "podfiltersbuttondatestr-news").click()
            print("Found and opened the timeline")

            time.sleep(8)  
        except NoSuchElementException:  
            print("Couldn't find the timeline open button")

    else: 
        print("Timeline is opened")

    timeline_opened = check_timeline_opened()
    print("Final check on timeline ")
    print(timeline_opened)
    time.sleep(6)
    return timeline_opened

#Function 2E: Check if timeline open or closed 
def check_timeline_opened():
    timeline_opened = False

    try:
        min_val_temp = driver.find_element(By.CSS_SELECTOR, ".min-val")   
        timeline_opened = True
        #print("The timeline is opened")
    except NoSuchElementException:  
        timeline_opened = False
        #print("The timeline is not opened")
    return timeline_opened

#STEP 3: Get Result Count and Toggle the Group Duplicates to On (DONE) 
def group_duplicates():
    print("STEP 3: Group Duplicates- Get Result Count and Toggle the Group Duplicates to On ") 

    #We have to trigger this first due to a small glitch that prevents the basin count from showing up 
    driver.find_element(By.CSS_SELECTOR, ".custom-control-indicator").click()
    time.sleep(5)
    
    #Step 1: Get both of the results with the Group Duplicates Toggled on and off
    basin_result_count_one_raw = get_result_count()
    driver.find_element(By.CSS_SELECTOR, ".custom-control-indicator").click()
    time.sleep(5)
    basin_result_count_two_raw = get_result_count()

    if type(basin_result_count_one_raw) == None:
        print("ERROR: The basin count was not available so quit")
        quit() 

#STEP 4: Set Sort by to Date (oldest to Newest)
def set_sort_by_date(): 
    print("STEP 4: Set Sort by to Date (oldest to Newest)")

    try:
        driver.find_element(By.ID, "sortbymenulabel").click()
        time.sleep(3)
    except NoSuchElementException:  
        print("NoSuchElementException couldn't find sortbymenulabel")

    try:
        driver.find_element(By.CSS_SELECTOR, "#dropdownmenu > button:nth-child(5)").click()
        time.sleep(3)
    except NoSuchElementException:  
        print("NoSuchElementException couldn't click dropdownmenu")

    time.sleep(2)
    print("STEP 4: FINISHED")

#STEP 5: Get Current Result Count
def get_result_count():
    print("STEP 5: Get Current Result Count")
    result_count = -1
    try:
        count_element = driver.find_element(By.CLASS_NAME, "countrendered")
        result_count = count_element.get_attribute('data-actualresultscount')
        time.sleep(1)
    except NoSuchElementException:  
        print("ERROR: Could not get the result count no element")
    
    print("STEP 5: FINISHED")

    return result_count

#STEP 6: Download Excel  
def download_excel(basin_code, min_raw, max_raw):
    min = str(min_raw)
    max = str(max_raw)
    print("STEP 6: Download Excel Files") 
    print("Starting Downloads: Excel files for ", min, ": from ", 0, " to ", max)

    download_start_stop = min + "-" + max
    download_file_name = "ResultsList_" + basin_code + "_202207_" + min + "_" + max

    driver.find_element(By.CSS_SELECTOR, ".has_tooltip:nth-child(1) > .la-Download").click()
    time.sleep(6)
    driver.find_element(By.ID, "ResultsListOnly").click()
    time.sleep(4)
    driver.find_element(By.ID, "XLSX").click()
    time.sleep(4)  

    #Send the total amount to download 
    driver.find_element(By.CSS_SELECTOR, ".nested:nth-child(3) #SelectedRange").click()
    time.sleep(4)
    driver.find_element(By.CSS_SELECTOR, ".nested:nth-child(3) #SelectedRange").send_keys(download_start_stop)
    time.sleep(8)

    #Type in the Filename 
    driver.find_element(By.ID, "FileName").click()
    time.sleep(3)
    driver.find_element(By.ID, "FileName").send_keys("fileName")
    driver.find_element(By.ID, "FileName").clear()
    time.sleep(3)
    driver.find_element(By.ID, "FileName").send_keys(download_file_name)
    time.sleep(3)

    driver.find_element(By.CSS_SELECTOR, ".button-group > .primary").click()
    #print("Function C1: Finished downloads from ", min, " to ", max)
    
    download_outcome = move_rename_file(download_file_name, download_folder_temp, download_file_name, download_folder)

    time.sleep(6)
    print("download_outcome")
    print(download_outcome)
    print("download_outcome")
    print("STEP 7: FINISHED")

    return True

#Function 6A: Move and Rename a File
def move_rename_file(original_file_name, original_file_path, new_file_name, new_file_path):
    download_wait_count = 0
    total_wait_seconds = 0
    #/Users/dvas22/Downloads/ResultsList_Aral_202207_1_220.ZIP
    
    while download_wait_count < 20:
        try:
            original_file_full = original_file_path + original_file_name + ".ZIP"
            new_file_full = new_file_path + new_file_name + ".ZIP"

            print("Original Download Location")
            print(original_file_full)

            print("Moving file to Geography Download Folder Location")
            print(new_file_full)

            os.rename(original_file_full, new_file_full)
            print("Step 6: The file was sucesfully moved")
            download_wait_count = 20
            time.sleep(5)
            return True

        except FileNotFoundError:
            download_wait_count = download_wait_count + 1
            total_wait_seconds = total_wait_seconds + 1
            print("Step 6: The file has not finished downloading yet pausing to sleep")
            print("Step 6: Total Wait ", total_wait_seconds * 5)
            wait_seconds(5)
    
    #Return true if the files were downloaded and moved 
    if download_wait_count < 20:
        return True
    else:
        return False

#FUNCTIONS B: Utility Functions 
#Function B1: Wait for period of seconds with no messages
def wait_seconds(total_wait_seconds):
    countdown_seconds = range(total_wait_seconds, 1, -1)

    for time_left in countdown_seconds:
        time.sleep(1)

#Function B2: Wait for period of seconds with messages
def wait_seconds_message(total_wait_seconds):
    countdown_seconds = range(total_wait_seconds, 1, -1)

    for time_left in countdown_seconds:
        print(time_left , "seconds left in wait period") 
        time.sleep(1)
    print("1 second left in wait period")
    print("")

#FUNCTIONS C: Excel Functions 
#Function C1: Update as success
def update_status_success(index, result_count, file_name):
    print("update_status_success")
    status_data.loc[index, ['finished']] = [1]
    status_data.loc[index, ['basin_count']] = [result_count]
    status_data.loc[index, ['file_name']] = [file_name]
    df = pd.DataFrame(status_data)  
    df.to_csv(status_file)

#Function C2: Update as Skipped
def update_status_failure(index, result_count):
    print("update_status_failure")
    status_data.loc[index, ['finished']] = [0]
    df = pd.DataFrame(status_data)  
    df.to_csv(status_file)

#Function C3: Update as Over Limit
def update_status_over_limit(index, result_count):
    print("update_status_over_limit")
    status_data.loc[index, ['finished']] = [1]
    status_data.loc[index, ['over_one_thousand']] = [1]
    status_data.loc[index, ['basin_count']] = [result_count]
    df = pd.DataFrame(status_data)  
    df.to_csv(status_file)


FileNotFoundError: [Errno 2] No such file or directory: '/Users/selenawallace/Documents/Data_Science/geography/status/excel/cngo.csv'

In [19]:
def main():
    
    #OSU Login
    #single_login() #this is for non Tufts users 
    
    #Tufts Login:
    #login_tufts_user(user_name)
    
    #Run Main Program
    single_basin_search() 
    time.sleep(60)


if __name__ == "__main__":
    main()

MAIN: Starting a single basin search for  gang
STEP 2: Set Date Range
Setting the date range from  01/01/2008  to  01/31/2008
Step 2A: Attempt to Open Timeline we will scroll to and try to click the button
Starting set_min_date
01/01/2008
Min date set
Starting set_max_date 
01/31/2008
Max date set


ElementClickInterceptedException: Message: element click intercepted: Element <button type="button" class="save disabled btn secondary" disabled="disabled">...</button> is not clickable at point (316, 656). Other element would receive the click: <div class="date-form">...</div>
  (Session info: chrome=125.0.6422.142)
Stacktrace:
0   chromedriver                        0x0000000108bce6b8 chromedriver + 6149816
1   chromedriver                        0x0000000108bc5cda chromedriver + 6114522
2   chromedriver                        0x0000000108652d5c chromedriver + 400732
3   chromedriver                        0x00000001086a6896 chromedriver + 743574
4   chromedriver                        0x00000001086a46c3 chromedriver + 734915
5   chromedriver                        0x00000001086a20e7 chromedriver + 725223
6   chromedriver                        0x00000001086a1437 chromedriver + 721975
7   chromedriver                        0x0000000108694672 chromedriver + 669298
8   chromedriver   

In [ ]:
#right now the issue is that it's not clickable
#so something about how the search resets...

In [ ]:
driver
dir(NoLinkClass)

['By',
 'EC',
 'NoLinkClass',
 'WebDriverWait',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'basin_code',
 'pd',
 'webdriver']